# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import ollama
from openai import OpenAI
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv
import os

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}

In [3]:
# set up environment

In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
# Get Llama 3.2 to answer
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling dde5aa3fc5ff... 100% ▕████████████████▏ 2.0 GB                         
pulling 966de95ca8a6... 100% ▕████████████████▏ 1.4 KB                         
pulling fcc5a6bec9da... 100% ▕████████████████▏ 7.7 KB                         
pulling a70ff7e570d9... 100% ▕████████████████▏ 6.0 KB                         
pulling 56bb8bd477a5... 100% ▕████████████████▏   96 B                         
pulling 34bb5ab01051... 100% ▕████████████████▏  561 B                         
verifying sha256 digest 
writing manifest 
success 


In [6]:
system_prompt = """You are a knowledgeable assistant that can receive various technical questions, \
                 and respond to those questions in a straightforward yet meticulous manner. \ 
                 Provide a reasonable explanation for why you think the answer is what you are providing.\ 
                 Try to be thorough in your answers, and avoid hallucinating. Provide your answer in Markdown.
                """

user_prompt = f"""Here is a technical question {question} \
                  Can you answer it and provide an explanation as well?
                """

In [7]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": user_prompt}
]

In [8]:
def stream_markdown(stream):
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [9]:
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
response = ollama_via_openai.chat.completions.create(
    model=MODEL_LLAMA,
    messages=messages, 
    stream=True
)

stream_markdown(response)

**Code Explanation**
---------------

python
yield from {
    "author": book.get("author")
    for book in books
    if book.get("author")
}

This code is using a combination of Python features to generate and yield values.

### Key Components:

1. **Generator Expression:** The code uses a generator expression, which is similar to an ordinary list comprehension but with the `yield` keyword instead of `return`. This allows it to produce a sequence of results that can be used in a loop without having to materialize all the values at once.
2. **Dictionary Comprehension:** Inside the generator expression, there's another dictionary comprehension, which is creating a new dictionary for each iteration of the outer loop (`for book in books`).
3. **`yield from` Statement:** The `yield from` statement is used to yield values from other iterators (in this case, dictionaries). It's similar to `yield`, but it delegates the iteration back to that other iterator.

**What Does it Do?**
-------------------

This code iterates over a list of books (`books`) and for each book, it checks if the book has an "author" key. If the book does have an author, its value is used as the content for this dictionary.

Ultimately, `yield from` then yields each of these dictionaries to an external function or loop, one by one. Each of these dictionaries will contain a single key-value pair: "author": author_name.

If the code were executed in isolation (not being called directly), it would print out the following:

json
{'author': 'author_0'},
{'author': 'author_1'},
{'author': 'author_2'}

Since this is an extension that's typically used with external functions or a main loop, its output depends on the function it is being called from.

Here is some example code to show these dictionary outputs

python
# Assume this is your data:
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": None},
    {"title": "Book 3", "author": "Author C"}
]

for author in yield from {book.get("author") for book in books if book.get("author")}:
    print(author)

**Answer Explanation**
-------------------

The `yield from` statement is an extension introduced in Python 3.3. It's used to simplify code that would require a normal loop and multiple separate function calls.

In this specific example, using `yield from` avoids creating a temporary list or other intermediate data structure, allowing the code to produce results more efficiently if working with large datasets.

Without this feature, you might write something like this:

python
def get_book_authors(books):
    for book in books:
        author = book.get("author")
        if author is not None:
            # yield is used here instead of return
            yield author

# Call this function and loop over its results.
# We have more flexibility on how we're passing the data to our function.

for author in get_book_authors(books):
    print(author)


The `yield from` version does exactly the same, but is usually simpler and less error-prone, especially when combined with dictionary comprehensions or other constructs.

In [10]:
# Get gpt-4o-mini to answer, with streaming

## Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [11]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

In [12]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.
response = openai.chat.completions.create(
        model = MODEL_GPT,
        messages = messages,
        stream=True
    )

# Step 4: print the result
stream_markdown(response)

Certainly! Let's break down the code you provided:

python
yield from {book.get("author") for book in books if book.get("author")}


### Explanation

1. **Set Comprehension**:
   - The code includes a **set comprehension** `{book.get("author") for book in books if book.get("author")}`. This means that it will create a set of authors from a collection called `books`. 
   - Each `book` is expected to be a dictionary-like object (e.g., a dictionary) that contains an `"author"` key.

2. **Iteration through `books`**:
   - The expression iterates over an iterable called `books`. This is typically a list or another collection of book objects, each of which is structured as a dictionary.

3. **Condition**:
   - The `if book.get("author")` part filters the collection. It checks if there's an `"author"` key in each `book` dictionary and whether its value is truthy (not `None`, `False`, or an empty string). If an author exists, it is included in the set.

4. **Getting Unique Authors**:
   - By using a set (`{...}`), this code collects unique authors. If multiple books have the same author, that author will only appear once in the resulting set, automatically filtering duplicates.

5. **Yielding**:
   - The `yield from` expression is a way to yield each item from an iterable. In this case, each unique author from the set comprehension will be yielded one at a time.
   - This construct is typically used within a generator function to yield values without needing to loop explicitly over the iterable.

### Final Result

Putting it all together, the provided code does the following:
- It collects all unique authors from the list of books where the `"author"` key is present and has a truthy value.
- Then, it yields those authors one by one.

### Example

Given the following list of books:

python
books = [
    {"title": "Book One", "author": "Author A"},
    {"title": "Book Two", "author": "Author B"},
    {"title": "Book Three", "author": "Author A"},
    {"title": "Book Four", "author": None},
    {"title": "Book Five"}
]


The expression would yield:

"Author A"
"Author B"


`"Author A"` appears only once in the output, despite having two entries in the `books` list.

### Summary
The line of code efficiently collects and yields unique author names from a list of books, while ignoring entries without an author or where the author is `None` or an empty string. This is useful in contexts where you want to process or display a list of authors without redundancy.